#  Rosenbrock function
---
Description:

- Optimization (min)
- Single-objective
- Constraints (1)
---

The general equation is given by:

- $f(x, y) = (1 - x)^2 + 100(y - x^2)^2$, with $-1.5 \le x \le +1.5$ and $-1.5 \le y \le +1.5$.

The problem here is that we are trying to minimize this function subject to the following constraint:

- $C_1(x, y) = x^2 + y^2 \le 2$.

To do this we apply the [Penalty method](https://en.wikipedia.org/wiki/Penalty_method). Within this setting the global minimum is found at:

$\hat{f}(1, 1) = 0$.

### First we import python libraries and set up the directory of our code.

In [61]:
import os, sys
import numpy as np

PROJECT_DIR = os.path.abspath('..')
sys.path.append(PROJECT_DIR)

### Here we import all our custom GA code.

In [62]:
# Import main classes.
from pygenalgo.genome.gene import Gene
from pygenalgo.genome.chromosome import Chromosome
from pygenalgo.engines.standard_ga import StandardGA
from pygenalgo.utils.utilities import cost_function

# Import Selection Operator(s).
from pygenalgo.operators.selection.linear_rank_selector import LinearRankSelector

# Import Crossover Operator(s).
from pygenalgo.operators.crossover.uniform_crossover import UniformCrossover

# Import Mutation Operator(s).
from pygenalgo.operators.mutation.random_mutator import RandomMutator

### Define the Rosenbrock function, which plays also the role of the 'fitness' function.

In addition, we define the 'rand_fx' which takes the role of the 'random()' method of the Genes. Every time we
want to 'mutate' a gene this function will be called that returns 'valid', but random values for the gene.

In [63]:
# Rosenbrock function.
@cost_function(minimize=True)
def fun_Rosenbrock(individual: Chromosome):
    
    # Penalty coefficient.
    rho = 10.0
    
    # Extract the data values as 'x' and 'y', for parsimony.
    x, y = individual.values()
    
    # Compute the function value.
    f1 = (1.0 - x)**2 + 100.0*(y - x**2)**2
    
    # Compute the constraint.
    C1 = max(0.0, x**2 + y**2 - 2.0)**2
    
    # Compute the final value.
    f_value = f1 + rho*C1

    # Condition for termination.
    solution_found = np.isclose(f1, 0.0, rtol=1.0e-5)
    
    # NOTE: the constraint is added with the penalty coefficient.
    return f_value, solution_found
# _end_def_

Here we set the GA parameters, such as number of genes, number of chromosomes, etc. Note that in this case each
gene has the same random() function (set by 'boundary_xy'). But if the problem demands otherwise it is easy to set a 
different random() function for each gene.

In [64]:
# Random number generator.
rng = np.random.default_rng()

# Random function: ~U(-1.5, +1.5).
boundary_xy = lambda: rng.uniform(-1.5, +1.5)

# Define the number of chromosomes.
N = 100

# Draw random samples for the initial points.
xy_init = rng.uniform(-1.5, +1.5, size=(N, 2))

# Initial population.
population = [Chromosome([Gene(xy_init[i, 0], boundary_xy),
                          Gene(xy_init[i, 1], boundary_xy)], np.nan, True)
              for i in range(N)]

# Create the StandardGA object that will carry on the optimization.
test_GA = StandardGA(initial_pop=population,
                     fit_func=fun_Rosenbrock,
                     select_op=LinearRankSelector(),
                     mutate_op=RandomMutator(),
                     crossx_op=UniformCrossover())

### Optimization process.

Here we call the GA object (either directly, or through the method run()). We set a number of parameter,
such as the maximum iterations (i.e. epochs), tolerance for the fitness convergences, etc.

In [65]:
test_GA(epochs=3000, f_tol=1.0e-6, elitism=True, adapt_probs=False)

Initial Avg. Fitness = -180.7058
Final   Avg. Fitness = -9.1670
Elapsed time: 8.477 seconds.


In [66]:
# Extract the optimal solution from the GA.
optimal_solution = test_GA.best_chromosome()

# Extract the data values as 'x' and 'y', for parsimony.
x_opt, y_opt = optimal_solution.values()

# Compute the function value.
f1_opt = (1.0 - x_opt)**2 + 100.0*(y_opt - x_opt**2)**2

# Display the (final) optimal value.
print(f"Minimum Found: {f1_opt:.5f}\n")
print(f"x = {x_opt:.5f}")
print(f"y = {y_opt:.5f}")

# True minimum: f(1.0, 1.0) = 0.0

Minimum Found: 0.00260

x = 0.94899
y = 0.90055


### End of file